# Setup

In [ ]:
print("\033[92mHello\033[0m")

Hello


In [ ]:
context_variables = {
    "student_data": """Here is the internal data of the student:
        1. NAME: Maximus Decimus Meridius
        2. CLASS: 10-A
        3. NOTES: problem with family
        """,
    "mood_history": ['happy', 'happy', 'neutral', 'sad', 'depressed', 'neutral', 'happy', 'sad'],
    "academic_info": """
        1. CLUB: Rugby
        2. PROFESSIONAL GOAL: Biologist
        3. VOLUNTEER: None
        4. GPA: 3.1
    """
}

https://stoic.tekloon.net/stoic-quote

In [ ]:
# Install the 'swarm' library from GitHub and the 'requests' library for HTTP requests
!pip install git+https://github.com/openai/swarm.git
!pip install requests

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-att18hpd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-att18hpd
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 29.2 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=038e1d168d272

In [ ]:
# Set OpenAI API Key
from google.colab import userdata
api_key = userdata.get('genai_course')

In [ ]:
# Import libraries
import os
from swarm import Swarm, Agent
import requests
from IPython.display import display, Markdown
import json
from bs4 import BeautifulSoup

In [ ]:
# Set the API key in the OS
os.environ["OPENAI_API_KEY"] = api_key

# Motivator AI Agent

In [ ]:
# Get the quote
def get_quote():
  return requests.get("https://stoic.tekloon.net/stoic-quote").json()

In [ ]:
# Define the AI Motivator Agent
motivator_agent = Agent(
    name="Motivator",
    instructions= "You motivate students with stoic motivational quotes, present the quotes and relate to the student's experience",
    functions = [get_quote]
)

In [ ]:
# Tranfering to the AI Motivator Agent
def transfer_to_motivator():
  return motivator_agent

# Counselor AI Agent

In [ ]:
def counselor(context_variables):
  student_info = context_variables.get("student_data", None)

  # Return detailed and friedly and approachable instructions
  return f"""
  You are a high school counselor specializing in helping the students improve their performance and achieve their goals.
  the student's record is {student_info}

  Provide advice and how to achieve their goals.

  Be friendly, supportive like a trusted advisor and mentor.
  Write concisely and to the point and keep the conversational natural.
  Avoid overwhelming lists or to-dos
  """

In [ ]:
# Build the counselor AI Agent
counselor_agent = Agent(
    name="Counselor",
    instructions= counselor(context_variables),
    functions = [transfer_back_to_therapist]
)

In [ ]:
# Build function to transfer to counselor
def transfer_to_counselor():
  return counselor_agent

# AI Crisis Handler

In [ ]:
# Defining its instructions
def crisis_handler():
  return """
  You are a high school counselor trained in managing crisis situations.
  Your goal is to provide reassurance ans support to the student.

  Write concisely and to the point and keep the conversational natural.
  Be empathetic and keep the student calm.
  Reassure the student and provide support.
  Mantain a warm, non-judgemental and supportive tone.
  """

In [ ]:
# Build the AI Crisis Agent
crisis_agent = Agent(
    name="Crisis Handler",
    instructions= crisis_handler()
)

In [ ]:
# Create a transfer to crisis handler
def transfer_to_crisis():
  return crisis_agent

# Therapist AI Agent

In [ ]:
# Build Therapist instructions
def therapist_instructions(context_variables):
  student_info = context_variables.get("student_data")
  student_mood_history = context_variables.get("mood_history")
  return f""" You are a therapist specializing in supporting high school stdents.
  Current student: {student_info}
  Your primary task is to consider their current mood history: {student_mood_history} and engae in conversations.
  IF the student is feeling down or unmotivated, call the motivator agent.
  If the student is focusing on their academic goals or their future education, call the counselor agent.
  If the student is feeling bad and you feel a human attention is required, call the crisis agent.
  You should be approachable, friently and encouraging.
  Write concisely and to the point and keep the conversational natural.
  Call one agent at a time.
  """

In [ ]:
# Main agent build
therapist_agent = Agent(
    name="Therapist",
    instructions= therapist_instructions(context_variables),
    functions = [transfer_to_motivator, transfer_to_counselor, transfer_to_crisis]
)

In [ ]:
# Define the back to therapist
def transfer_back_to_therapist():
  return therapist_agent

In [ ]:
# Append this funtion to the other AI agents
motivator_agent.functions.append(transfer_back_to_therapist)
crisis_agent.functions.append(transfer_back_to_therapist)

# Final AI Team of Agents

In [ ]:
# Build a function to improve the output
def pretty_print_messages(messages) -> None:
  # Iterate over the messages and skip those not from the assistant
  for message in messages:
    if message['role'] != "assistant":
      continue

    # Print the sender's name in blue
    print(f"\033[94m{message['sender']}\033[0m", end=": ")

    # if the message contains content, display it as markdown
    if message["content"]:
      display(Markdown(message["content"]))

    # Extract the tool calls
    #tool_calls = message.get("tool_calls", [])
    tool_calls = message.get("tool_calls") or []

    # Add a line if there multiple tool calls
    if len(tool_calls) > 1:
      print()

    # Iterate over all of the tools calls and print the details
    for tool_call in tool_calls:
      f = tool_call["function"]

      # Extract the name and arguments
      name, args = f["name"], f["arguments"]

      # Print the name and the arguments
      print(f"\033[95m{name}: \033[0m{args[1:-1]}")

In [ ]:
messages = []

# connect to OpenAI API
client = Swarm()
while True:
  user_input = input("\033[90mUser: \033[0m")

  messages.append({"role": "user", "content": user_input})

  if user_input.lower() == "bye":
    break

  response = client.run(
      agent = therapist_agent,
      messages = messages,
      context_variables= context_variables
  )

  # Display the output
  pretty_print_messages(response.messages)
  #print(display(Markdown(response.messages[-1]["content"])))

  # Append the messages
  messages.extend(response.messages)
  agent = response.agent


User: I am in a crisis
Therapist: transfer_to_crisis: 
Crisis Handler: 

I'm here to help you. Let's take a deep breath together. Can you tell me what's going on right now? You're not alone, and we'll figure this out together.

User: I am having an anxiety attack
Therapist: 

It's okay, Maximus. It's important to remember that anxiety attacks will pass, even though they feel overwhelming. Let’s try to do some calming exercises together:

1. **Deep Breathing:** Take slow, deep breaths. Inhale deeply through your nose for a count of four, hold for four, and then exhale slowly through your mouth for a count of four. Let's do this a few times.

2. **Grounding Exercise:** Look around you and name:
   - 5 things you can see
   - 4 things you can touch
   - 3 things you can hear
   - 2 things you can smell
   - 1 thing you can taste

Please take your time with these exercises, and if you feel like talking more about what's triggering this, I'm here to listen.

User: bye
